In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import yfinance as yf

from invaas.poptions import poptions
from matplotlib.pyplot import figure
from pandarallel import pandarallel

pd.options.mode.chained_assignment = None
pandarallel.initialize(nb_workers=12, progress_bar=True, verbose=2)

%matplotlib widget
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

data_directory = "data/spy-options-data-2020-2022/"
data_name = "spy_2020_2022"
raw_data_path = data_directory + f"{data_name}.csv"

periods = 20

In [ ]:
refresh_formatted_data = True

formatted_data_path = data_directory + f"{data_name}_formatted_{periods}.csv"

if not refresh_formatted_data and os.path.isfile(formatted_data_path):
    df_formatted = pd.read_csv(formatted_data_path, low_memory=False)
else:
    df_formatted = pd.read_csv(raw_data_path, low_memory=False)
    df_formatted["OPTION_ID"] = (
        df_formatted[" [EXPIRE_UNIX]"].astype(str) + df_formatted[" [STRIKE]"].astype(str)
    ).apply(lambda x: hash(x))
    df_formatted_option_id_count = df_formatted.groupby("OPTION_ID", as_index=False).size()
    df_formatted = df_formatted.loc[
        df_formatted.OPTION_ID.isin(
            df_formatted_option_id_count.loc[df_formatted_option_id_count["size"] >= periods].OPTION_ID.unique()
        )
    ]

    columns = df_formatted.columns
    columns = [s.replace("[", "") for s in columns]
    columns = [s.replace("]", "") for s in columns]
    columns = [s.replace(" ", "") for s in columns]
    df_formatted.columns = columns

    date_columns = ["QUOTE_READTIME", "QUOTE_DATE", "EXPIRE_DATE"]
    numeric_cols = df_formatted.columns.to_list()
    numeric_cols.remove("QUOTE_READTIME")
    numeric_cols.remove("QUOTE_DATE")
    numeric_cols.remove("EXPIRE_DATE")

    df_formatted.drop(columns=date_columns, inplace=True)

    for i in numeric_cols:
        df_formatted[i] = pd.to_numeric(df_formatted[i], errors="coerce")

    df_formatted.drop(columns=["C_SIZE", "P_SIZE"], inplace=True)
    df_formatted["EXPIRE_UNIX"] = pd.to_datetime(df_formatted.EXPIRE_UNIX, unit="s", utc=True)
    df_formatted["QUOTE_UNIXTIME"] = pd.to_datetime(df_formatted.QUOTE_UNIXTIME, unit="s", utc=True).apply(
        lambda x: pd.Timestamp(x).round(freq="D")
    )
    df_formatted["OPTION_ID"] = (df_formatted["EXPIRE_UNIX"].astype(str) + df_formatted["STRIKE"].astype(str)).apply(
        lambda x: hash(x)
    )

    df_formatted.sort_values(by=["QUOTE_UNIXTIME", "EXPIRE_UNIX", "STRIKE"], inplace=True)
    df_formatted_groupby_option = df_formatted.groupby("OPTION_ID", as_index=False)

    for column in ["C_ASK", "P_ASK"]:
        max_ask_column = f"MAX_{column}_{periods}"
        min_ask_column = f"MIN_{column}_{periods}"
        max_column_rename = {}
        min_column_rename = {}
        max_column_rename[column] = max_ask_column
        min_column_rename[column] = min_ask_column
        df_formatted.drop(columns=[max_ask_column, min_ask_column], errors="ignore", inplace=True)
        df_formatted_groupby_option_column_rolling = df_formatted_groupby_option[column].rolling(
            window=periods, min_periods=1
        )
        df_rolling_max_ask = (
            df_formatted_groupby_option_column_rolling.max()  # get index
            .rename(columns=max_column_rename)
            .drop(columns=["OPTION_ID"])
            .join(df_formatted)[["OPTION_ID", max_ask_column]]
            .groupby("OPTION_ID", as_index=False)
            .shift(periods=-periods)
            .join(df_formatted)[["OPTION_ID", max_ask_column]]
            .drop(columns=["OPTION_ID"])
        )
        df_rolling_min_ask = (
            df_formatted_groupby_option_column_rolling.min()  # get index
            .rename(columns=min_column_rename)
            .drop(columns=["OPTION_ID"])
            .join(df_formatted)[["OPTION_ID", min_ask_column]]
            .groupby("OPTION_ID", as_index=False)
            .shift(periods=-periods)
            .join(df_formatted)[["OPTION_ID", min_ask_column]]
            .drop(columns=["OPTION_ID"])
        )
        df_formatted = df_formatted.join(df_rolling_max_ask)
        df_formatted = df_formatted.join(df_rolling_min_ask)

    df_formatted.reset_index(inplace=True, drop=True)
    df_formatted.to_csv(formatted_data_path, index=False)

df_formatted

In [ ]:
refresh_processed_data = False

processed_data_path = data_directory + f"{data_name}_processed_{periods}.csv"

if not refresh_processed_data and os.path.isfile(processed_data_path):
    df_options = pd.read_csv(processed_data_path, low_memory=False)
else:

    def get_options_data(row):
        return [
            {
                "option_id": hash("call" + str(row.OPTION_ID)),
                "option_type": "call",
                "quote_unixtime": row.QUOTE_UNIXTIME,
                "expire_unixtime": row.EXPIRE_UNIX,
                "days_to_expire": row.DTE,
                "underlying_last": row.UNDERLYING_LAST,
                "strike": row.STRIKE,
                "strike_distance": row.STRIKE_DISTANCE,
                "strike_distance_pct": row.STRIKE_DISTANCE_PCT,
                "delta": row.C_DELTA,
                "gamma": row.C_GAMMA,
                "vega": row.C_VEGA,
                "theta": row.C_THETA,
                "rho": row.C_RHO,
                "iv": row.C_IV,
                "volume": row.C_VOLUME,
                "last": row.C_LAST,
                "bid": row.C_BID,
                "ask": row.C_ASK,
                "max_ask": row[f"MAX_C_ASK_{periods}"],
                "min_ask": row[f"MIN_C_ASK_{periods}"],
            },
            {
                "option_id": hash("put" + str(row.OPTION_ID)),
                "option_type": "put",
                "quote_unixtime": row.QUOTE_UNIXTIME,
                "expire_unixtime": row.EXPIRE_UNIX,
                "days_to_expire": row.DTE,
                "underlying_last": row.UNDERLYING_LAST,
                "strike": row.STRIKE,
                "strike_distance": row.STRIKE_DISTANCE,
                "strike_distance_pct": row.STRIKE_DISTANCE_PCT,
                "delta": row.P_DELTA,
                "gamma": row.P_GAMMA,
                "vega": row.P_VEGA,
                "theta": row.P_THETA,
                "rho": row.P_RHO,
                "iv": row.P_IV,
                "volume": row.P_VOLUME,
                "last": row.P_LAST,
                "bid": row.P_BID,
                "ask": row.P_ASK,
                "max_ask": row[f"MAX_P_ASK_{periods}"],
                "min_ask": row[f"MIN_P_ASK_{periods}"],
            },
        ]

    def flatten(xss):
        return [x for xs in xss for x in xs]

    options_data = flatten(df_formatted[: len(df_formatted)].parallel_apply(get_options_data, axis=1))
    df_options = pd.DataFrame(data=options_data)

    df_options.reset_index(inplace=True, drop=True)
    df_options.to_csv(processed_data_path, index=False)

df_options

In [ ]:
min_dte = 30
# max_dte = 30
max_ask = 1
min_ask = 0.1
min_volume = 1
max_strike_distance_pct = 0.2

df_options_filtered = df_options.loc[
    (df_options.bid > 0)
    & (df_options.ask <= max_ask)
    & (df_options.ask >= min_ask)
    & (df_options.volume >= min_volume)
    & (df_options.days_to_expire >= min_dte)
    # & (df_options.days_to_expire <= max_dte)
    & (df_options.strike_distance_pct <= max_strike_distance_pct)
]

# df_options_filtered = df_options.copy()

df_options_filtered["max_ask_diff_pct"] = (
    df_options_filtered.max_ask - df_options_filtered.ask
) / df_options_filtered.ask
df_options_filtered["min_ask_diff_pct"] = (
    df_options_filtered.min_ask - df_options_filtered.ask
) / df_options_filtered.ask

target_profit = 1

df_options_filtered["good_buy"] = (df_options_filtered.max_ask_diff_pct > target_profit) & (
    df_options_filtered.min_ask_diff_pct > -0.5
)

print(
    f"Good buy percentage: {len(df_options_filtered.loc[df_options_filtered.good_buy])/len(df_options_filtered)*100:.2f}%"
)

df_options_filtered_good_buy = df_options_filtered.loc[df_options_filtered.good_buy]
df_options_filtered_bad_buy = df_options_filtered.loc[~df_options_filtered.good_buy]

print(f"Total gain: ${df_options_filtered_good_buy.ask.sum() * target_profit * 100:.2f}")
# print(f"Total loss: ${df_options_filtered_bad_buy.ask.sum() - df_options_filtered_bad_buy.min_ask.sum():.2f}")
print(f"Total loss: ${df_options_filtered_bad_buy.ask.sum() * 0.25 * 100:.2f}")
display(df_options_filtered_good_buy.describe(include="all").astype(str))
display(df_options_filtered_bad_buy.describe(include="all").astype(str))